In [1]:
from matplotlib import style
import matplotlib.pyplot as plt 

In [2]:
import numpy as np
import pandas as pd

In [3]:
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [4]:
fires_df = pd.read_csv('Fires.csv')

In [5]:
fires_df

,Unnamed: 0,STATE,FIRE_YEAR,FIRE_SIZE,FIRE_SIZE_CLASS,STAT_CAUSE_DESCR,LATITUDE,LONGITUDE
0,0,CA,2005,0.10,A,Miscellaneous,40.036944,-121.005833
1,1,CA,2004,0.25,A,Lightning,38.933056,-120.404444
2,2,CA,2004,0.10,A,Debris Burning,38.984167,-120.735556
3,3,CA,2004,0.10,A,Lightning,38.559167,-119.913333
4,4,CA,2004,0.10,A,Lightning,38.559167,-119.933056
...,...,...,...,...,...,...,...,...
1880460,1880460,CA,2015,0.01,A,Missing/Undefined,40.481637,-122.389375
1880461,1880461,CA,2015,0.20,A,Miscellaneous,37.617619,-120.938570
1880462,1880462,CA,2015,0.10,A,Missing/Undefined,37.617619,-120.938570
1880463,1880463,CA,2015,2.00,B,Missing/Undefined,37.672235,-120.898356


In [6]:
fires_clean = fires_df[fires_df["STATE"] == "TX"]
fires_clean

,Unnamed: 0,STATE,FIRE_YEAR,FIRE_SIZE,FIRE_SIZE_CLASS,STAT_CAUSE_DESCR,LATITUDE,LONGITUDE
283,283,TX,2005,3.00,B,Arson,33.786111,-96.150000
284,284,TX,2005,55.00,C,Debris Burning,31.312500,-94.270833
285,285,TX,2005,2.00,B,Campfire,33.345833,-97.595833
286,286,TX,2005,0.30,B,Smoking,31.386667,-95.150556
287,287,TX,2005,55.00,C,Lightning,31.354167,-95.085278
...,...,...,...,...,...,...,...,...
1856360,1856360,TX,2015,49.70,C,Missing/Undefined,31.031333,-97.618000
1856399,1856399,TX,2015,1.00,B,Missing/Undefined,32.846100,-94.967600
1856439,1856439,TX,2015,0.25,A,Missing/Undefined,33.164283,-96.252667
1859961,1859961,TX,2015,250.00,D,Missing/Undefined,33.054000,-94.708667


In [7]:
fires_clean = fires_clean[['STATE', 'FIRE_YEAR', 'FIRE_SIZE', 'FIRE_SIZE_CLASS',
       'STAT_CAUSE_DESCR', 'LATITUDE', 'LONGITUDE']]

In [8]:
fires_clean.head()

,STATE,FIRE_YEAR,FIRE_SIZE,FIRE_SIZE_CLASS,STAT_CAUSE_DESCR,LATITUDE,LONGITUDE
283,TX,2005,3.0,B,Arson,33.786111,-96.150000
284,TX,2005,55.0,C,Debris Burning,31.312500,-94.270833
285,TX,2005,2.0,B,Campfire,33.345833,-97.595833
286,TX,2005,0.3,B,Smoking,31.386667,-95.150556
287,TX,2005,55.0,C,Lightning,31.354167,-95.085278


In [9]:
fires_clean_grouped_year = fires_clean.groupby(['FIRE_YEAR'])
fires_clean_grouped_year.count().head()

,STATE,FIRE_SIZE,FIRE_SIZE_CLASS,STAT_CAUSE_DESCR,LATITUDE,LONGITUDE
FIRE_YEAR,,,,,,
1992,1168,1168,1168,1168,1168,1168
1993,1457,1457,1457,1457,1457,1457
1994,907,907,907,907,907,907
1995,1648,1648,1648,1648,1648,1648
1996,2843,2843,2843,2843,2843,2843


In [10]:
fires_clean_grouped_year = fires_clean.groupby(['FIRE_YEAR'])

fires_per_year_df = pd.DataFrame(fires_clean_grouped_year['STATE'].count())
# fires_clean_grouped_year_df = pd.DataFrame(fires_clean_grouped_year['FIRE_YEAR'].count())

# fires_clean_grouped_year_df.head()

# fires_clean_grouped_year_df.to_csv('./Darrell/firenumber.csv')
fires_per_year_df

fires_per_year_clean_df = fires_per_year_df.rename(columns={"STATE": "FIRE_INCIDENTS"})
fires_per_year_clean_df


,FIRE_INCIDENTS
FIRE_YEAR,
1992,1168
1993,1457
1994,907
1995,1648
1996,2843
1997,770
1998,2026
1999,1775
2000,2627


In [22]:
fires_size_grouped_year_df = pd.DataFrame(fires_clean_grouped_year["FIRE_SIZE"].sum())
fires_size_byyear_df = fires_size_grouped_year_df


fires_size_byyear_df.head()
# fires_size_byyear_df.to_csv('./Darrell/firesize.csv')




,FIRE_SIZE
FIRE_YEAR,
1992,33610.3
1993,116873.7
1994,70917.5
1995,56371.1
1996,216223.0


In [35]:
fires_size_byyear_df['FIRE_SIZE'] = fires_size_byyear_df['FIRE_SIZE']

fires_size_byyear_df

,FIRE_SIZE
FIRE_YEAR,
1992,33610.3
1993,116873.7
1994,70917.5
1995,56371.1
1996,216223.0
1997,46780.9
1998,136551.4
1999,60028.2
2000,181686.8


In [12]:
inner_merge_df = pd.merge(fires_per_year_clean_df, fires_size_byyear_df, on="FIRE_YEAR")
inner_merge_df

inner_merge_df.to_csv('incidentsSizeMerged.csv')

In [13]:
fires_clean.to_csv('fires_clean2.csv', index=False)

In [14]:
fires_clean['STAT_CAUSE_DESCR'].unique()

array(['Arson', 'Debris Burning', 'Campfire', 'Smoking', 'Lightning',
       'Equipment Use', 'Miscellaneous', 'Children', 'Railroad',
       'Fireworks', 'Powerline', 'Missing/Undefined', 'Structure'],
      dtype=object)

In [15]:
drought_df = pd.read_csv('./Dillon/SPI-texas.csv')
drought_df = drought_df[drought_df.index > 1164]
drought_df = drought_df[drought_df.index < 1452]
drought_df

,0,DATE,D0,D1,D2,D3,D4,-9,W0,W1,W2,W3,W4
1165,1.0,d_19920201,0.0,0.0,0.0,0.0,0.0,0.0,99.1,95.6,81.1,63.9,35.6
1166,1.5,d_19920301,0.0,0.0,0.0,0.0,0.0,0.0,98.5,94.6,81.6,65.5,37.5
1167,2.9,d_19920401,0.0,0.0,0.0,0.0,0.0,0.0,97.1,94.1,78.1,58.0,24.0
1168,3.5,d_19920501,0.0,0.0,0.0,0.0,0.0,0.0,96.5,91.6,79.8,67.4,44.7
1169,2.4,d_19920601,0.0,0.0,0.0,0.0,0.0,0.0,97.5,92.2,79.4,63.6,33.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1447,3.7,d_20150801,0.0,0.0,0.0,0.0,0.0,0.0,96.4,87.7,59.2,38.4,16.4
1448,10.9,d_20150901,0.0,0.0,0.0,0.0,0.0,0.0,89.0,73.9,41.4,26.1,9.4
1449,3.9,d_20151001,0.0,0.0,0.0,0.0,0.0,0.0,96.1,85.4,56.4,38.6,20.8
1450,1.4,d_20151101,0.0,0.0,0.0,0.0,0.0,0.0,98.6,94.9,72.2,52.6,33.0


In [16]:
drought_df['DATE'] = drought_df['DATE'].map(lambda x: x.lstrip('d_').rstrip('d_'))
drought_df['DATE']
drought_df.to_csv('./Dillon/SPI-tx.csv')
# drought_df

In [17]:
drought_df

,0,DATE,D0,D1,D2,D3,D4,-9,W0,W1,W2,W3,W4
1165,1.0,19920201,0.0,0.0,0.0,0.0,0.0,0.0,99.1,95.6,81.1,63.9,35.6
1166,1.5,19920301,0.0,0.0,0.0,0.0,0.0,0.0,98.5,94.6,81.6,65.5,37.5
1167,2.9,19920401,0.0,0.0,0.0,0.0,0.0,0.0,97.1,94.1,78.1,58.0,24.0
1168,3.5,19920501,0.0,0.0,0.0,0.0,0.0,0.0,96.5,91.6,79.8,67.4,44.7
1169,2.4,19920601,0.0,0.0,0.0,0.0,0.0,0.0,97.5,92.2,79.4,63.6,33.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1447,3.7,20150801,0.0,0.0,0.0,0.0,0.0,0.0,96.4,87.7,59.2,38.4,16.4
1448,10.9,20150901,0.0,0.0,0.0,0.0,0.0,0.0,89.0,73.9,41.4,26.1,9.4
1449,3.9,20151001,0.0,0.0,0.0,0.0,0.0,0.0,96.1,85.4,56.4,38.6,20.8
1450,1.4,20151101,0.0,0.0,0.0,0.0,0.0,0.0,98.6,94.9,72.2,52.6,33.0


In [18]:
drought_df2 = pd.read_csv('./Dillon/PALEO-texas.csv')
drought_df2 = drought_df2[drought_df2.index > 1991]
drought_df2 = drought_df2[drought_df2.index < 2016]
drought_df2

,0,DATE,D0,D1,D2,D3,D4,-9,W0,W1,W2,W3,W4
1992,2.8,year_1992,0.0,0.0,0.0,0.0,0.0,0.0,97.3,86.8,73.5,36.9,9.1
1993,32.0,year_1993,2.7,0.4,0.0,0.0,0.0,0.0,65.3,47.7,22.4,7.7,1.2
1994,47.1,year_1994,32.9,7.6,0.8,0.0,0.0,0.0,20.0,7.0,0.0,0.0,0.0
1995,27.7,year_1995,25.7,12.0,3.0,0.0,0.0,0.0,46.7,26.1,7.1,0.3,0.0
1996,30.5,year_1996,67.4,40.1,6.6,0.0,0.0,0.0,2.0,0.4,0.0,0.0,0.0
1997,24.7,year_1997,0.4,0.0,0.0,0.0,0.0,0.0,75.0,46.7,19.6,3.9,0.0
1998,44.5,year_1998,45.7,24.2,3.9,0.0,0.0,0.0,9.8,2.6,0.0,0.0,0.0
1999,57.6,year_1999,15.6,4.5,0.4,0.0,0.0,0.0,26.8,8.6,1.1,0.0,0.0
2000,21.7,year_2000,72.6,52.4,24.0,1.9,0.0,0.0,5.7,1.9,0.0,0.0,0.0
2001,33.0,year_2001,30.7,18.4,8.1,0.0,0.0,0.0,36.4,14.7,6.6,2.0,0.4


In [19]:
drought_df2['DATE'] = drought_df2['DATE'].map(lambda x: x.lstrip('year_').rstrip('year_'))
drought_df2['DATE']
drought_df2.to_csv('./Dillon/PALEO-tx.csv')

In [20]:
drought_df2

,0,DATE,D0,D1,D2,D3,D4,-9,W0,W1,W2,W3,W4
1992,2.8,1992,0.0,0.0,0.0,0.0,0.0,0.0,97.3,86.8,73.5,36.9,9.1
1993,32.0,1993,2.7,0.4,0.0,0.0,0.0,0.0,65.3,47.7,22.4,7.7,1.2
1994,47.1,1994,32.9,7.6,0.8,0.0,0.0,0.0,20.0,7.0,0.0,0.0,0.0
1995,27.7,1995,25.7,12.0,3.0,0.0,0.0,0.0,46.7,26.1,7.1,0.3,0.0
1996,30.5,1996,67.4,40.1,6.6,0.0,0.0,0.0,2.0,0.4,0.0,0.0,0.0
1997,24.7,1997,0.4,0.0,0.0,0.0,0.0,0.0,75.0,46.7,19.6,3.9,0.0
1998,44.5,1998,45.7,24.2,3.9,0.0,0.0,0.0,9.8,2.6,0.0,0.0,0.0
1999,57.6,1999,15.6,4.5,0.4,0.0,0.0,0.0,26.8,8.6,1.1,0.0,0.0
2000,21.7,2000,72.6,52.4,24.0,1.9,0.0,0.0,5.7,1.9,0.0,0.0,0.0
2001,33.0,2001,30.7,18.4,8.1,0.0,0.0,0.0,36.4,14.7,6.6,2.0,0.4
